# Simulation experiment using noisy data 

Run entire simulation experiment multiple times to generate confidence interval.  The simulation experiment can be found in ```functions/pipeline.py```

In [1]:
%load_ext autoreload
%autoreload 2

from joblib import Parallel, delayed
import multiprocessing
import sys
import os
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings(action='ignore')

sys.path.append("../../")
from functions import pipelines, utils

from numpy.random import seed
randomState = 123
seed(randomState)

Using TensorFlow backend.


In [2]:
# Read in config variables
config_file = os.path.abspath(os.path.join(os.getcwd(),"../../configs", "config_Pa_sample_combat.tsv"))
params = utils.read_config(config_file)

In [3]:
# Load parameters
dataset_name = params["dataset_name"]
analysis_name = params["analysis_name"]
NN_architecture = params["NN_architecture"]
num_simulated_samples = params["num_simulated_samples"]
lst_num_experiments = params["lst_num_experiments"]
use_pca = params["use_pca"]
num_PCs = params["num_PCs"]
local_dir = params["local_dir"]
correction_method = params["correction_method"]

iterations = params["iterations"] 
num_cores = params["num_cores"]

In [4]:
# Additional parameters
file_prefix = "Experiment"
corrected = False

In [5]:
# Input file
base_dir = os.path.abspath(
      os.path.join(
          os.getcwd(), "../.."))

normalized_data_file = os.path.join(
    base_dir,
    dataset_name,    
    "data",
    "input",
    "train_set_normalized.pcl")

In [6]:
# Output files
similarity_uncorrected_file = os.path.join(
    base_dir,
    "results",
    "saved_variables",
    dataset_name + "_sample_lvl_sim_similarity_uncorrected_"+correction_method+".pickle")

ci_uncorrected_file = os.path.join(
    base_dir,
    "results",
    "saved_variables",
    dataset_name + "_sample_lvl_sim_ci_uncorrected_"+correction_method+".pickle")

similarity_permuted_file = os.path.join(
    base_dir,
    "results",
    "saved_variables",
    dataset_name + "_sample_lvl_sim_permuted")

In [7]:
# Run multiple simulations
results = Parallel(n_jobs=num_cores, verbose=100)(
    delayed(
        pipelines.sample_level_simulation_uncorrected)(i,
                                                       NN_architecture,
                                                       dataset_name,
                                                       analysis_name,
                                                       num_simulated_samples,
                                                       lst_num_experiments,
                                                       corrected,
                                                       use_pca,
                                                       num_PCs,
                                                       file_prefix,
                                                       normalized_data_file,
                                                       local_dir) for i in iterations)

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   1 tasks      | elapsed: 56.3min
[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed: 56.4min remaining: 84.6min
[Parallel(n_jobs=5)]: Done   3 out of   5 | elapsed: 56.5min remaining: 37.6min
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed: 56.6min remaining:    0.0s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed: 56.6min finished


In [8]:
# permuted score
permuted_score = results[0][0]

In [9]:
# Concatenate output dataframes
all_svcca_scores = pd.DataFrame()

for i in iterations:
    all_svcca_scores = pd.concat([all_svcca_scores, results[i][1]], axis=1)

all_svcca_scores

,score,score,score,score,score
number of experiments,,,,,
1,0.999994,0.999994,0.999994,0.999994,0.999994
2,0.900667,0.900439,0.900757,0.900788,0.900367
5,0.610618,0.608025,0.609659,0.607602,0.610798
10,0.132702,0.130952,0.129850,0.130990,0.127041
20,0.045943,0.048769,0.044716,0.042069,0.040250
50,0.148028,0.150653,0.146515,0.146074,0.145222
100,0.236035,0.239522,0.236452,0.245729,0.241058
500,0.725593,0.724178,0.730950,0.746450,0.727755
1000,0.938411,0.937955,0.932476,0.936408,0.929687


In [10]:
# Get mean svcca score for each row (number of experiments)
mean_scores = all_svcca_scores.mean(axis=1).to_frame()
mean_scores.columns = ['score']
mean_scores

,score
number of experiments,
1,0.999994
2,0.900604
5,0.609340
10,0.130307
20,0.044349
50,0.147298
100,0.239759
500,0.730985
1000,0.934987


In [11]:
# Get standard dev for each row (number of experiments)
import math
std_scores = (all_svcca_scores.std(axis=1)/math.sqrt(10)).to_frame()
std_scores.columns = ['score']
std_scores

,score
number of experiments,
1,1.430770e-08
2,6.007888e-05
5,4.639728e-04
10,6.613508e-04
20,1.051311e-03
50,6.747200e-04
100,1.246890e-03
500,2.850468e-03
1000,1.193179e-03


In [12]:
# Get confidence interval for each row (number of experiments)
# z-score for 95% confidence interval 
err = std_scores*1.96

In [13]:
# Get boundaries of confidence interval
ymax = mean_scores + err
ymin = mean_scores - err

ci = pd.concat([ymin, ymax], axis=1)
ci.columns = ['ymin', 'ymax']
ci

,ymin,ymax
number of experiments,,
1,0.999994,0.999994
2,0.900486,0.900721
5,0.608431,0.610250
10,0.129011,0.131603
20,0.042289,0.046410
50,0.145976,0.148621
100,0.237315,0.242203
500,0.725398,0.736572
1000,0.932649,0.937326


In [14]:
mean_scores

,score
number of experiments,
1,0.999994
2,0.900604
5,0.609340
10,0.130307
20,0.044349
50,0.147298
100,0.239759
500,0.730985
1000,0.934987


In [15]:
# Pickle dataframe of mean scores scores for first run, interval
mean_scores.to_pickle(similarity_uncorrected_file)
ci.to_pickle(ci_uncorrected_file)
np.save(similarity_permuted_file, permuted_score)